🔹 Step C — Punto di partenza

Adesso sei messo così:

Hai un corpus di 8323 giocatori, ognuno con una descrizione testuale text_profile.

Hai costruito un indice TF-IDF (Step B) con:

tfidf_vectorizer.joblib

tfidf_matrix.joblib

index_metadata.csv

Hai già una funzione search_players(query, top_k) che prende la query “così com’è” e calcola la similarità coseno sui profili.

👉 Con Step C vogliamo aggiungere uno strato di intelligenza sulla query:

capire se l’utente cerca un ruolo (playmaker / guardia / ala / centro),

capire se parla di skill specifiche (tiratore da 3, rimbalzista offensivo, difensore, passatore…),

riscrivere/arricchire la query in modo da dare più peso a questi concetti chiave quando la trasformiamo in TF-IDF.

In [4]:
import os
import re
import numpy as np
import pandas as pd
import joblib

from sklearn.metrics.pairwise import cosine_similarity

# ============================
# 1. Caricamento indice esistente
# ============================

MODELS_DIR = "../data/models"
VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH = os.path.join(MODELS_DIR, "index_metadata.csv")

vectorizer = joblib.load(VECTORIZER_PATH)
tfidf_matrix = joblib.load(MATRIX_PATH)
metadata_df = pd.read_csv(META_PATH)

print("Indice caricato.")
print("TF-IDF matrix shape:", tfidf_matrix.shape)
print("Metadata shape:", metadata_df.shape)

# ============================
# 2. Normalizzazione
# ============================

def normalize_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text


# ============================
# 3. Regole di parsing semantico
# ============================

# Skill principali: tiri, rimbalzi, difesa, playmaking
SKILL_PATTERNS = {
    "shooting_3": [
        "tiratore da 3", "ottime percentuali da 3", "buon tiratore da 3",
        "da 3 punti", "tripla", "3 punti"
    ],
    "ft": [
        "buon ft", "ottimo ft", "buon tiratore di liberi", "ottimo tiratore di liberi",
        "buon ft%", "buon ft", "tiri liberi", "liberi"
    ],
    "reb": [
        "rimbalzista", "forte rimbalzista", "buon rimbalzista", "rimbalzi",
        "dominante a rimbalzo", "forte a rimbalzo"
    ],
    "playmaking": [
        "playmaker", "buon passatore", "assist", "creatore di gioco",
        "regista", "fa girare la squadra"
    ],
    "scorer": [
        "realizzatore", "scorer", "tanti punti", "tanti punti a partita",
        "prima opzione offensiva"
    ]
}

# Espansione di testo che useremo per ogni concetto (parole che esistono nei profili)
SEMANTIC_EXPANSIONS = {
    "shooting_3": "ottimo tiratore da 3 3 punti 3P% buon tiratore da 3",
    "ft": "tiri liberi FT% buon tiratore ai liberi eccellente tiratore ai liberi",
    "reb": "rimbalzi forte rimbalzista buon rimbalzista",
    "playmaking": "assist buon passatore ottimo playmaker",
    "scorer": "segna molti punti realizzatore punti a partita"
}

# Peso (quante volte ripetere l’espansione nel testo pesato)
SEMANTIC_WEIGHTS = {
    "shooting_3": 4,
    "ft": 3,
    "reb": 3,
    "playmaking": 3,
    "scorer": 3
}

#Status del giocatore, se è in attività o meno
STATUS_PATTERNS = {
    "Active": ["in attività", "ancora in attività", "ancora gioca", "sta ancora giocando"],
    "Retired": ["ritirato", "non gioca più", "ha smesso", "in pensione"]
}

# ============================
# 4. Parsing della query -> intenti
# ============================

def parse_query_intents(raw_query):
    """
    Dato l'input naturale dell'utente, rileva:
    - set di skill richieste (shooting_3, reb_off, difesa, playmaking, ...)
    Restituisce un dict con:
      {
        "normalized_query": ...,
        "skills": [...],
      }
    """
    q = normalize_text(raw_query)
    detected_skills = set()
    detected_status = None

    # Stato (attivo / ritirato)
    # Stato
    for status, patterns in STATUS_PATTERNS.items():
        for p in patterns:
            if p in q:
                detected_status = status

        
      # Skill
    for skill, patterns in SKILL_PATTERNS.items():
        for p in patterns:
            if p in q:
                detected_skills.add(skill)
                break

    return {
        "normalized_query": q,
        "skills": sorted(list(detected_skills)),
        "status": detected_status
    }

# ============================
# 5. Costruzione query pesata
# ============================

def build_weighted_query(raw_query, intents):
    base = intents["normalized_query"]
    boosted = [base]

    for skill in intents["skills"]:
        exp = SEMANTIC_EXPANSIONS.get(skill, "")
        w = SEMANTIC_WEIGHTS.get(skill, 1)
        boosted.append((" " + exp) * w)

    return " ".join(boosted).strip()

# ============================
# 6. Motore di ricerca "intelligente"
# ============================

def search_players_scout(query: str, top_k = 10):
    """
    Versione avanzata della search:
    - fa parsing semantico della query
    - costruisce una query pesata
    - calcola similarità coseno contro i profili
    - restituisce risultati + info sugli intenti interpretati
    """
    if not query or not isinstance(query, str):
        raise ValueError("La query deve essere una stringa non vuota.")

    # 1) Parsing semantico
    intents = parse_query_intents(query)

    # 2) Costruzione query pesata
    weighted_query = build_weighted_query(query, intents)

    # 3) TF-IDF della query pesata
    q_vec = vectorizer.transform([weighted_query])
    sim_scores = cosine_similarity(q_vec, tfidf_matrix).ravel()

    # 4) Ordina
    top_idx = np.argsort(sim_scores)[::-1]
    results = metadata_df.iloc[top_idx].copy()
    results["similarity"] = sim_scores[top_idx]

    # FILTRO PER STATUS (Active / Retired)
    if intents["status"] and "Status" in results.columns:
        results = results[results["Status"] == intents["status"]]

    return results.head(top_k), intents, weighted_query

# ============================
# 7. Test 
# ============================

query_test = "cerco un buon tiratore da 3 e buon passatore ancora in attività"

results, intents, wq = search_players_scout(query_test)

print("Query:", query_test)
print("\nIntenti interpretati:")
print(" - Skills:", intents.get("skills"))
print(" - Status rilevato:", intents.get("status"))   # <<< QUI LA NOVITÀ
print(" - Query normalizzata:", intents.get("normalized_query"))
print("\nQuery pesata (prime 200 lettere):")
print(wq[:200], "...")

cols = ["Player", "DraftYear", "Pick", "similarity", "Status", "text_profile"]
cols = [c for c in cols if c in results.columns]

print("\nTOP risultati:")
display(results[cols])



Indice caricato.
TF-IDF matrix shape: (8323, 22025)
Metadata shape: (8323, 7)
Query: cerco un buon tiratore da 3 e buon passatore ancora in attività

Intenti interpretati:
 - Skills: ['playmaking', 'shooting_3']
 - Status rilevato: Active
 - Query normalizzata: cerco un buon tiratore da 3 e buon passatore ancora in attività

Query pesata (prime 200 lettere):
cerco un buon tiratore da 3 e buon passatore ancora in attività  assist buon passatore ottimo playmaker assist buon passatore ottimo playmaker assist buon passatore ottimo playmaker  ottimo tiratore d ...

TOP risultati:


,Player,DraftYear,Pick,similarity,Status,text_profile
8311,Antonio Reeves,2024,47,0.273275,Active,Giocatore: Antonio Reeves. Proveniente da Kent...
7680,Doug McDermott,2014,11,0.251932,Active,Giocatore: Doug McDermott. Proveniente da Crei...
8316,Quinten Post,2024,52,0.251321,Active,Giocatore: Quinten Post. Proveniente da Boston...
8302,Ajay Mitchell,2024,38,0.239306,Active,Giocatore: Ajay Mitchell. Proveniente da UC Sa...
7730,Karl-Anthony Towns,2015,1,0.208791,Active,Giocatore: Karl-Anthony Towns. Proveniente da ...
8228,Dariq Whitehead,2023,22,0.198411,Active,Giocatore: Dariq Whitehead. Proveniente da Duk...
8054,Payton Pritchard,2020,26,0.194744,Active,Giocatore: Payton Pritchard. Proveniente da Or...
7930,Grayson Allen,2018,21,0.190449,Active,Giocatore: Grayson Allen. Proveniente da Duke....
7861,Luke Kennard,2017,12,0.190224,Active,Giocatore: Luke Kennard. Proveniente da Duke. ...
7798,Jakob Poeltl,2016,9,0.184421,Active,Giocatore: Jakob Poeltl. Proveniente da Utah. ...


In questo step la ricerca non è più una semplice TF-IDF(query) vs TF-IDF(text_profile): ora il sistema interpreta la query dell’utente.
Un motore rule-based riconosce:

le skill richieste (tiratore da 3, buon ai liberi, rimbalzista, passatore, realizzatore…);

lo status del giocatore richiesto (in attività / ritirato).

La query viene poi riscritta e potenziata con termini realmente presenti nei profili, es.
“ottimo tiratore da 3 punti tiro da 3 3P%” oppure “assist buon passatore”.

Questo rende i concetti chiave molto più influenti nella similarità coseno, migliorando drasticamente la pertinenza dei risultati.

Il sistema diventa così un primo motore intelligente da talent scout, capace di capire l’intento dell’utente e orientare la ricerca. Nel prossimo step aggiungerai anche spiegazioni testuali dei risultati.